In [1]:
!pip install -q webdataset

In [2]:
import os
import json
import multiprocessing
from pathlib import Path

import polars as pl
import webdataset as wds
from tqdm import tqdm
import joblib
import pandas as pd
from webdataset import TarWriter

from graphnet.data.constants import FEATURES, TRUTH


graphnet: INFO     2023-03-14 20:48:45 - get_logger - Writing log to logs/graphnet_20230314-204845.log


In [3]:
meta_data_path = Path("../../raw/icecube-neutrinos-in-deep-ice/train_meta.parquet")
geometry_path = Path("../../raw/icecube-neutrinos-in-deep-ice/sensor_geometry.csv")
input_data_path = "../../raw/icecube-neutrinos-in-deep-ice/train/batch_{batch_id}.parquet"
shard_dir = Path("./webdatasets")
shard_dir.mkdir(exist_ok=True)

shard_filename = str(shard_dir / 'shards-%04d.tar')

shard_size = int(1 * 1024**3)

In [3]:
meta_data = pl.read_parquet(meta_data_path)
geometry_table = pl.read_csv(geometry_path, dtypes={"sensor_id": pl.Int16})

In [4]:
def get_features_truth(event_id):
    truth = meta_data_batch.filter(pl.col("event_id") == event_id)[TRUTH.KAGGLE].to_numpy()
    features = (
        df_batch.filter(pl.col("event_id") == event_id)
            .join(geometry_table, on="sensor_id", how="left")[FEATURES.KAGGLE]
            .to_numpy()
    )
    return features, truth

In [5]:
def write_one_batch(batch_id, meta_data, geometry_table,):
    pattern = "webdatasets/batch-%03d.tar"
    print(f"Start saving batch {batch_id}")
    meta_data_batch = meta_data[meta_data.batch_id == batch_id]
    event_ids = meta_data_batch["event_id"].unique()
    df_batch = pd.read_parquet(input_data_path.format(batch_id=batch_id))
    df_batch[["x", "y", "z"]] = geometry_table.loc[df_batch.sensor_id.values, ["x", "y", "z"]].values

    fname = pattern % batch_id
    stream = TarWriter(fname)
    
    
    for event_id in tqdm(event_ids, desc=f"Batch {batch_id}"):
        df_event = df_batch[df_batch.index == event_id].copy()
        write_samples_into_single_shard(stream, meta_data_batch, event_id, df_event)

    print(f"Finished saving batch {batch_id}")
    stream.close()


def write_samples_into_single_shard(stream, meta_data_batch, event_id, df_batch):
    truth = meta_data_batch[meta_data_batch.event_id == event_id][TRUTH.KAGGLE].values
    features = df_batch.loc[event_id, FEATURES.KAGGLE].values

    data = {
            "__key__": str(event_id),
            "pickle": (
                features, truth,
            )
        }

    size = stream.write(data)
    return size

In [ ]:

with wds.ShardWriter(shard_filename, maxsize=shard_size,) as sink:
    for batch_id in batch_ids:
        meta_data_batch = meta_data.filter(pl.col("batch_id") == batch_id)
        event_ids = meta_data_batch["event_id"].unique()
        df_batch = pl.read_parquet(input_data_path.format(batch_id=batch_id))

        for event_id in tqdm(event_ids, desc="fBatch {batch_id}"):
            features, truth = get_features_truth(event_id)
            sink.write({
                "__key__": str(event_id),
                "pickle": (
                    features, truth,
                )
            })
            
        del df_batch, meta_data_batch


In [7]:
batch_ids = range(101, 151)

meta_data = pd.read_parquet(meta_data_path)
geometry_table = pd.read_csv(geometry_path) # dtypes={"sensor_id": pl.Int16}
geometry_table = geometry_table.set_index("sensor_id")


In [8]:
for batch_id in batch_ids:
    write_one_batch(batch_id, meta_data, geometry_table)

Start saving batch 101


Batch 101: 100%|██████████████████████████████████████████| 200000/200000 [1:39:29<00:00, 33.50it/s]


Finished saving batch 101
Start saving batch 102


Batch 102: 100%|██████████████████████████████████████████| 200000/200000 [1:40:49<00:00, 33.06it/s]


Finished saving batch 102
Start saving batch 103


...

Finished saving batch 148
Start saving batch 149


Batch 149: 100%|██████████████████████████████████████████| 200000/200000 [1:39:46<00:00, 33.41it/s]


Finished saving batch 149
Start saving batch 150


Batch 150: 100%|██████████████████████████████████████████| 200000/200000 [1:45:22<00:00, 31.63it/s]

Finished saving batch 150
